# Practice time

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

In [ ]:
os.getcwd() # shows the current folder / directory you are in

## Preliminary task - change to appropriate folder

In [ ]:
# change directory to Preprocess folder
os.chdir('put/path/to/folder/here') #

In [ ]:
# Load the dataset
df = pd.read_csv('customer_data_test.csv')

### INITIAL EXPLORATION

In [ ]:
# Inspect first rows
print("\nFirst 10 rows:")
print(df.head(10))

In [ ]:
# Check data types
print("\nData types:")
print(df.info())

In [ ]:
# Summary statistics
print("\nSummary statistics:")
print(df.describe())


In [ ]:
# Missing values
print("\nMissing values count:")
print(df.isnull().sum())
print("\nMissing values percentage:")
print(df.isnull().sum() / len(df) * 100)

### IDENTIFYING AND REMOVING DUPLICATES

In [ ]:
print(f"\nOriginal dataset shape: {df.shape}")
print(f"Number of duplicate rows: {df.duplicated().sum()}")

In [ ]:
# Show some duplicate examples
duplicates = df[df.duplicated(keep=False)].sort_values('CustomerID')
print("\nExample of duplicate records:")
print(duplicates.head(10))

In [ ]:
# Remove duplicates
df_clean = df.drop_duplicates().copy()
print(f"\nAfter removing duplicates: {df_clean.shape}")

### FIXING DATA TYPES

In [ ]:
# Fix Income column (mixed types)
print("\nIncome column has mixed types (some stored as strings)")
print(f"Current dtype: {df_clean['Income'].dtype}")


In [ ]:
# Convert to numeric (strings will be converted, non-numeric become NaN)
df_clean['Income'] = pd.to_numeric(df_clean['Income'], errors='coerce')
print(f"After conversion: {df_clean['Income'].dtype}")

In [ ]:
# Fix DatePurchased column
print("\nDatePurchased column has mixed types")
print(f"Current dtype: {df_clean['DatePurchased'].dtype}")
df_clean['DatePurchased'] = pd.to_datetime(df_clean['DatePurchased'], errors='coerce')
print(f"After conversion: {df_clean['DatePurchased'].dtype}")

### DETECTING OUTLIERS

In [ ]:
# Outliers in TransactionAmount using IQR method
print("\nDetecting outliers in TransactionAmount using IQR method:")
Q1 = df_clean['TransactionAmount'].quantile(0.25)
Q3 = df_clean['TransactionAmount'].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [ ]:
print(f"Q1: {Q1:.2f}")
print(f"Q3: {Q3:.2f}")
print(f"IQR: {IQR:.2f}")
print(f"Lower bound: {lower_bound:.2f}")
print(f"Upper bound: {upper_bound:.2f}")

In [ ]:
outliers_transaction = df_clean[(df_clean['TransactionAmount'] < lower_bound) | 
                                (df_clean['TransactionAmount'] > upper_bound)]
print(f"\nNumber of outliers: {len(outliers_transaction)}")
print("\nOutlier values:")
print(outliers_transaction[['CustomerID', 'TransactionAmount']].head(10))

In [ ]:
# Outliers in Income using Z-score method
print("\n\nDetecting outliers in Income using Z-score method:")
income_mean = df_clean['Income'].mean()
income_std = df_clean['Income'].std()
df_clean['Income_Zscore'] = (df_clean['Income'] - income_mean) / income_std

In [ ]:
outliers_income = df_clean[abs(df_clean['Income_Zscore']) > 3]
print(f"Number of outliers (|Z-score| > 3): {len(outliers_income)}")
print("\nOutlier values:")
print(outliers_income[['CustomerID', 'Income', 'Income_Zscore']].head(10))


### VISUALIZING OUTLIERS WITH BOXPLOTS

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplot for TransactionAmount
axes[0].boxplot(df_clean['TransactionAmount'].dropna())
axes[0].set_title('Boxplot: Transaction Amount', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Amount (£)')
axes[0].grid(True, alpha=0.3)

# Boxplot for Income
axes[1].boxplot(df_clean['Income'].dropna())
axes[1].set_title('Boxplot: Income', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Income (£)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
# plt.savefig('outliers_boxplots.png', dpi=300, bbox_inches='tight')
# print("\nBoxplots saved as 'outliers_boxplots.png'")
plt.show()
plt.close()


### TREATING OUTLIERS

In [ ]:
# Option 1: Remove outliers
df_no_outliers = df_clean[(df_clean['TransactionAmount'] >= lower_bound) & 
                          (df_clean['TransactionAmount'] <= upper_bound)]
print(f"\nOption 1 - Remove outliers: {df_clean.shape} -> {df_no_outliers.shape}")

In [ ]:
# Option 2: Winsorize (clip) outliers
df_winsorized = df_clean.copy()
df_winsorized['TransactionAmount_Winsorized'] = df_winsorized['TransactionAmount'].clip(
    lower=lower_bound, upper=upper_bound)
print(f"\nOption 2 - Winsorized: Values clipped to [{lower_bound:.2f}, {upper_bound:.2f}]")

In [ ]:

# Option 3: Replace with median
df_median_replace = df_clean.copy()
median_transaction = df_clean['TransactionAmount'].median()
df_median_replace.loc[(df_median_replace['TransactionAmount'] < lower_bound) | 
                     (df_median_replace['TransactionAmount'] > upper_bound), 
                     'TransactionAmount'] = median_transaction
print(f"\nOption 3 - Replace with median: Outliers replaced with {median_transaction:.2f}")

In [ ]:

# Option 4: Flag outliers
df_flagged = df_clean.copy()
df_flagged['Is_Outlier'] = ((df_flagged['TransactionAmount'] < lower_bound) | 
                             (df_flagged['TransactionAmount'] > upper_bound))
print(f"\nOption 4 - Flag outliers: Added 'Is_Outlier' column")
print(f"Flagged {df_flagged['Is_Outlier'].sum()} outliers")

### HANDLING MISSING VALUES

In [ ]:
print("\nMissing values summary:")
missing_summary = pd.DataFrame({
    'Column': df_clean.columns,
    'Missing_Count': df_clean.isnull().sum(),
    'Missing_Percentage': df_clean.isnull().sum() / len(df_clean) * 100
})
print(missing_summary[missing_summary['Missing_Count'] > 0])

In [ ]:
# Option 1: Remove rows with missing values (only if small and random)
df_dropped = df_clean.dropna()
print(f"\nOption 1 - Drop rows: {df_clean.shape} -> {df_dropped.shape}")


In [ ]:
# Option 2: Impute with mean/median for numeric
df_imputed = df_clean.copy()


df_imputed['Age'] = df_imputed['Age'].fillna(df_imputed['Age'].median())
df_imputed['Income'] = df_imputed['Income'].fillna(df_imputed['Income'].median())

print(f"\nOption 2 - Impute numeric: Age and Income filled with median")
print(f"Remaining missing values: {df_imputed[['Age', 'Income']].isnull().sum().sum()}")


In [ ]:
# Option 3: Impute categorical with mode or placeholder
df_imputed['Feedback'] = df_imputed['Feedback'].fillna('No feedback provided')

print(f"\nOption 3 - Impute categorical: Feedback filled with 'No feedback provided'")
print(f"Remaining missing in Feedback: {df_imputed['Feedback'].isnull().sum()}")


In [ ]:
# Option 4: Replace with 0 (context-dependent)
df_zero = df_clean.copy()
df_zero['TransactionAmount'] = df_zero['TransactionAmount'].fillna(0)

print(f"\nOption 4 - Replace with 0: For fields where 0 is meaningful")

### FIXING INCONSISTENCIES

In [ ]:
# Check for inconsistent categories
print("\nCategory values (before cleaning):")
print(df_imputed['Category'].value_counts())

# Standardize categories
df_imputed['Category'] = df_imputed['Category'].str.lower().str.strip()
df_imputed['Category'] = df_imputed['Category'].replace({
    'electronics': 'electronics',
    'food items': 'food',
    'clothing': 'clothing'
})

print("\nCategory values (after cleaning):")
print(df_imputed['Category'].value_counts())


### CLEANING TEXT

In [ ]:
# Clean Feedback text
df_imputed['Feedback_Original'] = df_imputed['Feedback']

# Lowercase
df_imputed['Feedback_Clean'] = df_imputed['Feedback'].str.lower()

# Remove punctuation
df_imputed['Feedback_Clean'] = df_imputed['Feedback_Clean'].str.replace('[^\w\s]', '', regex=True)

# Strip whitespace
df_imputed['Feedback_Clean'] = df_imputed['Feedback_Clean'].str.strip()

# Word count
df_imputed['Feedback_WordCount'] = df_imputed['Feedback_Clean'].str.split().str.len()

print("\nText cleaning example:")
print(df_imputed[['Feedback_Original', 'Feedback_Clean', 'Feedback_WordCount']].head(10))


### FINAL CLEANED DATASET

In [ ]:
# Select final columns
df_final = df_imputed[['CustomerID', 'Age', 'Income', 'TransactionAmount', 
                       'Category', 'ProductName', 'Feedback_Clean', 'DatePurchased']].copy()

In [ ]:
print(f"\nFinal dataset shape: {df_final.shape}")
print("\nMissing values in final dataset:")
print(df_final.isnull().sum())

In [ ]:
print("\nFinal summary statistics:")
print(df_final.describe())

In [ ]:
# Save cleaned dataset if needed - what should be done?
# df_final.to_csv('cleaned_customer_data.csv', index=False)
# print("\nCleaned dataset saved as 'cleaned_customer_data.csv'")

## VISUALIZATION OF DATA - BEFORE AND AFTER

In [ ]:

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histogram of TransactionAmount - Before
axes[0, 0].hist(df_clean['TransactionAmount'], bins=30, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Transaction Amount - With Outliers', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Amount (£)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(True, alpha=0.3)

# Histogram of TransactionAmount - After
axes[0, 1].hist(df_no_outliers['TransactionAmount'], bins=30, edgecolor='black', alpha=0.7)
axes[0, 1].set_title('Transaction Amount - Without Outliers', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Amount (£)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].grid(True, alpha=0.3)

# Missing values - Before
missing_before = df.isnull().sum()
missing_before = missing_before[missing_before > 0]
axes[1, 0].barh(missing_before.index, missing_before.values, color='coral')
axes[1, 0].set_title('Missing Values - Before Cleaning', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Count')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# Missing values - After
missing_after = df_final.isnull().sum()
if missing_after.sum() > 0:
    missing_after = missing_after[missing_after > 0]
    axes[1, 1].barh(missing_after.index, missing_after.values, color='lightgreen')
else:
    axes[1, 1].text(0.5, 0.5, 'No Missing Values!', ha='center', va='center', 
                    fontsize=16, fontweight='bold', transform=axes[1, 1].transAxes)
axes[1, 1].set_title('Missing Values - After Cleaning', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Count')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
# plt.savefig('before_after_comparison.png', dpi=300, bbox_inches='tight')
# print("\nComparison plots saved as 'before_after_comparison.png'")
plt.show()
plt.close()



In [ ]:
print("\n" + "="*80)
print("DATA CLEANING COMPLETE!")
print("="*80)
print("\nKey takeaways:")
print("1. Always explore data before cleaning (.head(), .info(), .describe())")
print("2. Check for and remove duplicates")
print("3. Fix data types early")
print("4. Detect outliers using boxplots, IQR, or Z-scores")
print("5. Decide outlier treatment based on context")
print("6. Handle missing values appropriately (remove, impute, or flag)")
print("7. Fix inconsistencies in categorical variables")
print("8. Clean text fields (lowercase, remove punctuation)")
print("9. Document and justify all cleaning decisions")
print("10. Verify cleaned data before analysis")


## Not covered today

In [ ]:
print("Merging datasets")